# Notebook 3 : Apprentissage non-supervisé

Notebook préparé par [Chloé-Agathe Azencott](http://cazencott.info).

Dans ce notebook il s'agit d'explorer plusieurs techniques de réduction de dimension et de clustering.

In [ ]:
# charger numpy as np, matplotlib as plt
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

In [ ]:
import pandas as pd

## 1. Analyse en composantes principales

Dans cette section, nous allons effectuer une analyse en composantes principales d'un jeu de données décrivant les scores obtenus par les meilleurs athlètes ayant participé en 2004 à une épreuve de décathlon, aux Jeux Olympiques d'Athènes ou au Décastar de Talence.

### Chargement des données

Les données sont contenues dans le fichier `decathlon.txt`.

Le fichier contient 42 lignes et 13 colonnes.

La première ligne est un en-tête qui décrit les contenus des colonnes.

Les lignes suivantes décrivent les 41 athlètes.

Les 10 premières colonnes contiennent les scores obtenus aux différentes épreuves.
La 11ème colonne contient le classement.
La 12ème colonne contient le nombre de points obtenus.
La 13ème colonne contient une variable qualitative qui précise l'épreuve (JO ou Décastar) concernée.

Nous allons examiner ces données avec la librairie `pandas`.

In [ ]:
my_data = pd.read_csv('data/decathlon.txt', sep="\t")  # lire les données dans un dataframe

__Alternativement :__ Si vous avez besoin de télécharger le fichier (par exemple sur colab) :

In [ ]:
my_data.head()

### Visualisation

Une __matrice de nuages de points__ est une visualisation en k x k panneaux des relations deux à deux entre k variables :
* sur la diagonale, l'histogramme pour chacune des variables 
* hors diagonale, les nuages de points entre deux variables (non standardisées).

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.plotting.scatter_matrix.html

Par exemple :

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(my_data[['Shot.put','High.jump', '400m']], alpha=0.5, s=60,
               figsize=(8, 8));

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(my_data, alpha=0.5, s=60,
               figsize=(18, 18));

Alternativement, la librairie `seaborn` permet des visualisations plus élaborées que `matplotlib`. Vous pouvez par exemple explorer les capacités de `jointplot`. 
https://seaborn.pydata.org/generated/seaborn.jointplot.html

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')

sns.jointplot(x='Shot.put', y='400m', data = my_data, 
              height=6, space=0, color='b')

sns.jointplot(x='Shot.put', y='400m', data = my_data, 
              kind='reg', height=6, space=0, color='b')

Nous allons maintenant effectuer une analyse en composantes principales des scores aux 10 épreuves.

Commençons par extraire les données :

In [ ]:
X = np.array(my_data.drop(columns=['Points', 'Rank', 'Competition']))
print(X.shape)

### Standardisation des données

In [ ]:
from sklearn import preprocessing

In [ ]:
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

### Calcul des composantes principales

Les algorithmes de factorisation de matrice de `scikit-learn` sont inclus dans le module `decomposition`. Pour  l'ACP, référez-vous à : 
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
from sklearn import decomposition

Remarque : Nous avons ici peu de variables et pouvons nous permettre de calculer toutes les PC. 

La plupart des algorithmes implémentés dans `scikit-learn` suivent le fonctionnement suivant : 
* on instancie un objet, correspondant à un type d'algorithme/modèle, avec ses hyperparamètres (ici le nombre de composantes)
* on utilise la méthode `fit` pour passer les données à cet algorithme
* les paramètres appris sont maintenant accessibles comme arguments de cet objet.

In [ ]:
# Instanciation d'un objet PCA pour 10 composantes principales
pca = decomposition.PCA(n_components=10)

In [ ]:
# On passe maintenant les données standardisées à cet objet
# C'est ici que se font les calculs
pca.fit(X_scaled)

### Proportion de variance expliquée par les PCs

Nous allons maintenant afficher la proportion de variance expliquée par les différentes composantes. Il est accessible dans le paramètre `explained_variance_ration_` de notre objet `pca`.

In [ ]:
plt.plot(np.arange(1, 11), pca.explained_variance_ratio_, marker='o')

plt.xlabel("Nombre de composantes principales")
plt.ylabel("Proportion de variance expliquée")

Nous pouvons aussi afficher la proportion *cumulative* de variance expliquée

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
plt.plot(np.arange(1, 11), np.cumsum(pca.explained_variance_ratio_), marker='o')

plt.xlabel("Nombre de composantes principales")
plt.title("Proportion cumulative de variance expliquée")

On peut aussi calculer la proportion de variance expliquée par les 4 (par exemple) premières PC avec

In [ ]:
print("%.2f" % np.sum(pca.explained_variance_ratio_[:4]))

__Questions :__ 
* Quelle est la proportion de variance expliquée par les deux premières composantes ? 
* Combien de composantes faudrait-il utiliser pour expliquer 80% de la variance des données ?

In [ ]:
print("%.2f" % np.sum(pca.explained_variance_ratio_[:2]))

### Projection des données sur les deux premières composantes principales

Nous allons maintenant utiliser uniquement les deux premières composantes principales.

Commençons par calculer la nouvelle représentation des données, c'est-à-dire leur projection sur ces deux PC. 

In [ ]:
pca = decomposition.PCA(n_components=2)
pca.fit(X_scaled)
X_projected = pca.transform(X_scaled)
print(X_projected.shape)

On peut afficher un nuage de points représentant les données selon ces deux PC.

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1])

plt.xlabel("PC 1")
plt.ylabel("PC 2")

On peut maintenant colorer chaque point du nuage de points ci-dessus en fonction du classement de l'athlète qu'il représente. 

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1], c=my_data['Rank'])

plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.colorbar(label='classement')

__Question :__ Qu'en conclure sur l'interprétation de la PC1 ?

### Interprétation des deux composantes principales
Chaque composante principale est une combinaison linéaire des variables décrivant les données. Les poids de cette combinaison linéaire sont accesibles dans `pca.components_`.

Nous pouvons visualiser non pas les individus, mais les 10 variables dans l'espace des 2 composantes principales.

In [ ]:
pcs = pca.components_
print(pcs[0])

In [ ]:
fig = plt.figure(figsize=(6, 6))

plt.scatter(pcs[0], pcs[1])
for (x_coordinate, y_coordinate, feature_name) in zip(pcs[0], pcs[1], my_data.columns[:10]):
    plt.text(x_coordinate, y_coordinate, feature_name)                          
    
plt.xlabel("Contribution à la PC1")
plt.ylabel("Contribution à la PC2")

__Question :__ Quelles variables ont des contributions très similaires aux deux composantes principales ? Qu'en déduire sur leur similarité ?

__Question :__ Comment interpréter le signe des contributions des variables à la première composantes principales ?

## 2. Données « Olivetti »

Nous allons maintenant utiliser la réduction de dimension pour représenter en deux dimensions un jeu de données contenant des visages. Il s'agit d'un jeu de données classique, contenant 400 photos de 64 par 64 pixels. Il s'agit de photos des visages de 40 personnes différentes (10 photos par personne), étiquetées par un numéro de classe entre 0 et 39 identifiant la personne.

Nous pouvons charger ce jeu de données directement grâce à scikit-learn :

In [ ]:
from sklearn import datasets

In [ ]:
data = datasets.fetch_olivetti_faces()

__Si vous n'arrivez pas à télécharger les données :__
* Aller sur : https://github.com/CroncLee/PCA-face-recognition/blob/master/olivetti_py3.pkz
* Télécharger le fichier (bouton Download) 
utiliser la commande
```
    data = datasets.fetch_olivetti_faces(data_home="<PATH TO DATA>")
```
En remplaçant <PATH TO DATA> par le chemin vers le dossier où vous avez enregistré les données.

In [ ]:
X = data.data
y = data.target

In [ ]:
print(X.shape)

In [ ]:
print("Les données contiennent %d classes" % len(np.unique(y)))

Chaque image est représentée par une valeur (niveau de gris) pour chacun de ses pixels. 

Nous pouvons visualiser ces images à condition de réorganiser ces valeurs (= un vecteur de longueur 4096) en matrices 64x64. Par exemle ci-dessous pour l'image à l'index 23 :

In [ ]:
plt.imshow(X[13, :].reshape((64, 64)), interpolation='nearest', cmap=plt.cm.gray)

### PCA

Commençons par une analyse en composantes principales comme à la section précédente :

In [ ]:
pca = decomposition.PCA(n_components=2)
X_transformed_pca = pca.fit_transform(X)

Chaque image est maintenant représentée par non pas 4096 variables, mais par deux variables. Nous pouvons les visualiser en nuage de point, et les colorer par classe :

In [ ]:
plt.scatter(X_transformed_pca[:, 0], X_transformed_pca[:, 1], 
            c=y)
plt.xlabel("Première composante principale")
plt.ylabel("Deuxième composante principale")

__Question :__ les images du même visage (= de la même classe) ont-elles des représentations proches ?

Nous pouvons visualiser la contribution de chaque pixel à la première composante principale :

In [ ]:
plt.imshow(pca.components_[0, :].reshape((64,64)), interpolation='nearest', cmap=plt.cm.gray)

Puis la contribution de chaque pixel à la deuxième composante principale :

In [ ]:
plt.imshow(pca.components_[1, :].reshape((64,64)), interpolation='nearest', cmap=plt.cm.gray)

### tSNE

Nous allons maintenant utiliser la même démarche, mais avec tSNE, grâce à la classe [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) du module `manifold`.

In [ ]:
from sklearn import manifold

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', learning_rate='auto')

In [ ]:
%%time 
X_transformed = tsne.fit_transform(X)

In [ ]:
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], 
            c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE")

#### Influence du paramètre de perplexité 

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', learning_rate='auto', perplexity=1)

In [ ]:
%%time 
X_transformed = tsne.fit_transform(X)

In [ ]:
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], 
            c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE")

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', learning_rate='auto', perplexity=100)

In [ ]:
%%time 
X_transformed = tsne.fit_transform(X)

In [ ]:
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], 
            c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE")

## 3. Cercles imbriqués

Générons un jeu de données en deux dimensions formé de deux cercles imbriqués :

In [ ]:
from sklearn import datasets

In [ ]:
# nombre de points
n_samples = 1500

# set random seed
np.random.seed(37)

circles_X, circles_labels = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05)

Visualisons ces données :

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=circles_labels)

Supposons maintenant ne pas disposer des étiquettes. Quels algorithmes de clustering permettent de trouver deux clusters, correspondant chacun à un des cercles ?

### Algorithme des k-moyennes

L'objectif de l'algorithme k-means est retrouver $K$ clusters (et leur centroïde $\mu_k$) de manière à **minimiser la variance intra-cluster** :

\begin{align}
V = \sum_{k = 1}^{K} \sum_{x \in C_k} \frac{1}{|C_k|} (\|x - \mu_k\|^2)
\end{align}

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
from sklearn import cluster

In [ ]:
# initialisation d'un k-means avec k=2
kmeans = cluster.KMeans(n_clusters=2)

# application aux données 
kmeans.fit(circles_X)

L'attribut `.labels_` contient, pour chaque observation, le numéro du cluster auquel cette observation est assignée.

In [ ]:
kmeans.labels_

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=kmeans.labels_)
plt.title("Clustering K-means (K=2)")

#### Trouver K avec le coefficient de silhouette

Le coefficient (ou score) de silhouette permet de **comparer les distances moyennes intra- et inter-cluster** :

\begin{align}
\text{score} = \frac{b - a}{\max(a, b)}
\end{align}

avec $a$ la distance moyenne intra-cluster et $b$ la distance d'un point au cluster étranger le plus proche. Le score se calcule par observation (avec une valeur entre -1 et 1) puis la moyenne de ce score permet d'évaluer le clustering du nuage de point dans son ensemble.

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score

In [ ]:
from sklearn import metrics

In [ ]:
print("Coefficient de silhouette pour le k-means (k=2) : %.2f" % metrics.silhouette_score(circles_X, kmeans.labels_))

In [ ]:
silhouettes = []
k_values = range(2, 9)
for kval in k_values:
    kmeans_k = cluster.KMeans(n_clusters=kval)
    kmeans_k.fit(circles_X)
    silhouettes.append(metrics.silhouette_score(circles_X, kmeans_k.labels_))

In [ ]:
plt.plot(k_values, silhouettes)

plt.xlabel("K")
plt.ylabel("silhouette")

print("Coefficient de silhouette du KMeans en fonction de K")

In [ ]:
best_silhouette = np.max(silhouettes)
print("Coefficient de silhouette optimal : %.2f" % best_silhouette)
best_K = k_values[silhouettes.index(best_silhouette)]
print("K correspondant : %.2f" % best_K)

In [ ]:
kmeans_k = cluster.KMeans(n_clusters=best_K)
kmeans_k.fit(circles_X)

fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=kmeans_k.labels_)
plt.title("Clustering K-means (K=%d)" % best_K)

### DBSCAN (Clustering par densité)

L'algorithme DBSCAN (Density-Based Spatial Clustering of Applications with Noise) fonctionne en deux temps :
- Toutes les observations suffisamment proches sont connectées entre elles.
- Les observations avec un nombre minimal de voisins connectés sont considérées comme des *core samples*, à partir desquelles les clusters sont étendues. **Toutes les observations suffisamment proche d'un *core sample* appartiennent au même cluster que celui-ci**. 

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

In [ ]:
# initialisation d'un clustering DBSCAN
dbscan = cluster.DBSCAN(eps=0.2, min_samples=2)

# application aux données 
dbscan.fit(circles_X)

In [ ]:
np.unique(dbscan.labels_)

L'attribut `.labels_` contient, pour chaque observation, le numéro du cluster auquel cette observation est assignée.

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=dbscan.labels_)
plt.title("Clustering DBSCAN (eps=0.2)")

#### Rôle du paramètre de taille de voisinage (`eps`)

Si `eps` est trop petit :

In [ ]:
# initialisation d'un clustering DBSCAN
dbscan_005 = cluster.DBSCAN(eps=0.05, min_samples=2)

# application aux données 
dbscan_005.fit(circles_X)

In [ ]:
np.unique(dbscan_005.labels_)

L'attribut `.labels_` contient, pour chaque observation, le numéro du cluster auquel cette observation est assignée.

In [ ]:
fig = plt.figure(figsize=(5, 5))

outliers = np.where(dbscan_005.labels_ == -1)[0]
plt.scatter(circles_X[outliers, 0], circles_X[outliers, 1], marker='*', color='red')

non_outliers = np.where(dbscan_005.labels_ != -1)[0]
plt.scatter(circles_X[non_outliers, 0], circles_X[non_outliers, 1], c=dbscan_005.labels_[non_outliers])
plt.title("Clustering DBSCAN (eps=0.05)")

Si `eps` est trop grand :

In [ ]:
# initialisation d'un clustering DBSCAN
dbscan_2 = cluster.DBSCAN(eps=2., min_samples=2)

# application aux données 
dbscan_2.fit(circles_X)

In [ ]:
np.unique(dbscan_2.labels_)

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=dbscan_2.labels_)
plt.title("Clustering DBSCAN (eps=2.)")

#### Trouver eps avec le coefficient de silhouette

In [ ]:
print("Coefficient de silhouette pour DBSCAN (eps=0.2) : %.2f" % metrics.silhouette_score(circles_X, dbscan.labels_))

In [ ]:
eps_values = np.logspace(-3, 1, 40)
silhouettes = []

for eps in eps_values:
    dbscan_eps = cluster.DBSCAN(eps=eps, min_samples=2)
    dbscan_eps.fit(circles_X)
    if len(np.unique(dbscan_eps.labels_)) > 1: # nécessaire pour calculer le coeff de silhouette
        silhouettes.append(metrics.silhouette_score(circles_X, dbscan_eps.labels_))
    else:
        silhouettes.append(-1)

In [ ]:
plt.plot(eps_values, silhouettes)
plt.xscale("log")
plt.xlabel("eps (échelle log)")
plt.ylabel("silhouette")

In [ ]:
best_silhouette = np.max(silhouettes)
print("Coefficient de silhouette optimal : %.2f" % best_silhouette)
print("Eps correspondant : %.2f" % eps_values[silhouettes.index(best_silhouette)])

### Index de Rand ajusté

L'index de Rand ajusté permet de **comparer un résultat de clustering avec des étiquettes**. Pour chaque paire d'observations nous regardons si elles se situent dans le même cluster ou non, dans le clustering prédit et réel. L'index prend des valeurs entre 0 (clustering aléatoire) et 1 (clustering parfait).

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html

__Question :__ Pourquoi ne pas utiliser une métrique d'évaluation de modèle de classification ici ?

In [ ]:
print("Index de Rand ajusté du K-means (K=2) : %.2f" % metrics.adjusted_rand_score(circles_labels, kmeans.labels_))

In [ ]:
print("Index de Rand ajusté de dbscan (eps=0.2) : %.2f" % metrics.adjusted_rand_score(circles_labels, dbscan.labels_))

## 2. Manchots

On reprend ici les données utilisées dans le notebook 4

In [ ]:
palmerpenguins = pd.read_csv("data/penguins_data.csv")

__Alternativement :__ Si vous avez besoin de télécharger le fichier (par exemple sur colab) :

In [ ]:
!wget https://raw.githubusercontent.com/CBIO-mines/fml-dassault-systems/main/data/penguins_data.csv

palmerpenguins = pd.read_csv("penguins_data.csv")

In [ ]:
palmerpenguins = palmerpenguins[palmerpenguins['bill_depth_mm'].notna()]
palmerpenguins = palmerpenguins.reset_index()

In [ ]:
penguins_X = np.array(palmerpenguins[["bill_length_mm", "body_mass_g"]])

In [ ]:
from sklearn import preprocessing

In [ ]:
# standardisation (centrer-réduire)
penguins_X = preprocessing.StandardScaler().fit_transform(penguins_X)

In [ ]:
species_names, species_int = np.unique(palmerpenguins.species, return_inverse=True)
penguins_labels = species_int

In [ ]:
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels)
plt.xlabel("bill_length_mm (centrée-réduite)")
plt.ylabel("body_mass_g (centrée-réduite)")

### KMeans

In [ ]:
# initialisation d'un k-means avec k=3
kmeans = cluster.KMeans(n_clusters=3)

# application aux données 
kmeans.fit(penguins_X)

In [ ]:
#plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels, marker='o')
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=kmeans.labels_, marker='*')


plt.xlabel("bill_length_mm (centrée-réduite)")
plt.ylabel("body_mass_g (centrée-réduite)")

In [ ]:
print("Coefficient de silhouette pour le k-means (k=3) : %.2f" % metrics.silhouette_score(penguins_X, kmeans.labels_))

In [ ]:
print("Index de Rand ajusté du K-means (K=3) : %.2f" % metrics.adjusted_rand_score(penguins_labels, kmeans.labels_))

### DBSCAN

In [ ]:
eps_values = np.logspace(-3, 1, 40)
silhouettes = []

for eps in eps_values:
    dbscan_eps = cluster.DBSCAN(eps=eps, min_samples=2)
    dbscan_eps.fit(penguins_X)
    if len(np.unique(dbscan_eps.labels_)) > 1: # nécessaire pour calculer le coeff de silhouette
        silhouettes.append(metrics.silhouette_score(penguins_X, dbscan_eps.labels_))
    else:
        silhouettes.append(-1)

In [ ]:
plt.plot(eps_values, silhouettes)
plt.xscale("log")
plt.xlabel("eps (échelle log)")
plt.ylabel("silhouette")

In [ ]:
best_silhouette = np.max(silhouettes)
print("Coefficient de silhouette optimal : %.2f" % best_silhouette)
best_eps = eps_values[silhouettes.index(best_silhouette)]
print("Eps correspondant : %.2f" % best_eps)

In [ ]:
dbscan_opt = cluster.DBSCAN(eps=best_eps, min_samples=2)
dbscan_opt.fit(penguins_X)

In [ ]:
np.unique(dbscan_opt.labels_)

In [ ]:
print("Index de Rand ajusté de DBSCAN : %.2f" % metrics.adjusted_rand_score(penguins_labels, dbscan_opt.labels_))

In [ ]:
#plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels, marker='o')
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=dbscan_opt.labels_, marker='*')


plt.xlabel("bill_length_mm (centrée-réduite)")
plt.ylabel("body_mass_g (centrée-réduite)")

### Modèle de mélange gaussien 

Le modèle de mélange de gaussiennes cherche à **optimiser les paramètres d'un nombre fini de gaussiennes** aux données. 

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html

In [ ]:
from sklearn import mixture

In [ ]:
# initialisation d'un k-means avec k=3
gmm = mixture.GaussianMixture(n_components=3)

# application aux données 
gmm.fit(penguins_X)

# prédiction des clusters
gmm_labels = gmm.predict(penguins_X)

In [ ]:
#plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels, marker='o')
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=gmm_labels, marker='*')


plt.xlabel("bill_length_mm (centrée-réduite)")
plt.ylabel("body_mass_g (centrée-réduite)")

In [ ]:
print("Coefficient de silhouette pour le GMM (k=3) : %.2f" % metrics.silhouette_score(penguins_X, gmm_labels))

In [ ]:
print("Index de Rand ajusté du GMM (K=3) : %.2f" % metrics.adjusted_rand_score(penguins_labels, gmm_labels))